<b>Starting with library importing</b>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout,SpatialDropout1D
from tensorflow.keras.layers import Embedding

In [ ]:
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
df=pd.read_csv("Hotel_Reviews.csv",parse_dates=['Review_Date'],nrows=20000)
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

<b>Extracting the reviews on positive and negative data</b

In [ ]:
df['review']=df['Negative_Review']+df['Positive_Review']
df['review']

In [ ]:
df['review_type']=df['Reviewer_Score'].apply(lambda x:"negative" if x<7 else "positive")
df['review_type']

In [ ]:
df['Reviewer_Score'].unique()

In [ ]:
df['review_type'].value_counts()

In [ ]:
df=df[['review','review_type']]
df

In [ ]:
df

In [ ]:
sns.countplot(
  x='review_type',
  data=df,
  order=df.review_type.value_counts().index
)
plt.xlabel("Type")
plt.title("Review type")

In [ ]:
positive_review=df[df.review_type=='positive']
negative_review=df[df.review_type=='negative']

In [ ]:
positive_df=positive_review.sample(n=len(negative_review),random_state=RANDOM_SEED)
negative_df=negative_review

In [ ]:
review_df=pd.concat([positive_df,negative_df]).reset_index(drop=True)

In [ ]:
review_df.shape

In [ ]:
review_df.head()

In [ ]:
sns.countplot(
  x='review_type',
    data=review_df,
    order=review_df.review_type.value_counts().index
)
plt.xlabel("type")
plt.title("Review type(resampled)")

In [ ]:
review_df['review_type'].value_counts()

In [ ]:
sentiment_label=review_df.review_type.factorize()
sentiment_label

In [ ]:
review=review_df.review.values
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(review)
vocab_size=len(tokenizer.word_index)+1
encoded_docs=tokenizer.texts_to_sequences(review)
padded_sequences=pad_sequences(encoded_docs,maxlen=200)

In [ ]:
print(review[2])
print(encoded_docs[2])

In [ ]:
embedding_vector_length=32
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_length,input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50,dropout=0.5,recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
history=model.fit(padded_sequences,sentiment_label[0],validation_split=0.2,epochs=5,batch_size=32)

In [ ]:
def predict_sentiment(text):
    tw=tokenizer.texts_to_sequences([text])
    tw=pad_sequences(tw,maxlen=200)
    predict=int(model.predict(tw).round().item())
    print("Predicted label:",sentiment_label[1][predict])

In [ ]:
test_sentence1='This hotel is fantastic'
predict_sentiment(test_sentence1)

test_sentence2='This hotel is not my type'
predict_sentiment(test_sentence2)

In [ ]:
model.save_weights('model.h5')